# import the necessary package

In [2]:
import pandas as pd
import pickle
import random
pd.set_option('display.max_columns',60)
%matplotlib inline


# load the dataset

**News Articles ** 

In [3]:
newsData = pd.read_pickle('../Data/Feedly_Processed_DF_cleaned.pkl')

## Labeled Data 

### bank tagging

In [7]:
bankLabel = pd.read_csv('../Data/Labeled_Data/banks.csv')

In [8]:
bankLabel.head()

,article_id,published,title,url,feed_label,Bank1,Bank2
0,10f9ed2,2018-01-11,ADB Provides Support for Three Infrastructure ...,http://moderndiplomacy.eu/2018/01/11/adb-provi...,NEWS ADB - All Streams,adb,NaN
1,c0eece9b,2018-05-13,ADB Helps Inaugurate New Power Distribution Ne...,http://feedproxy.google.com/~r/adb_news/~3/2My...,NEWS ADB - All Streams,adb,NaN
2,d1d79dd8,2018-02-20,ADB Provides $360 Million for Rolling Stock to...,http://feedproxy.google.com/~r/adb_news/~3/v9s...,NEWS ADB - All Streams,adb,NaN
3,f0d65e5,2018-02-25,ADB provides financing to Thailand's B.Grimm P...,https://www.dealstreetasia.com/stories/adb-b-g...,NEWS ADB - All Streams,adb,NaN
4,4a557358,2018-02-26,ADB's $235m loan to support B.Grimm Power expa...,https://www.power-technology.com/news/adbs-235...,NEWS ADB - All Streams,adb,NaN


In [9]:
newsData.head()

,article_id,title,url,feed_label,content,published,summary,article_text,article_keywords,article_text_len,top_lang
10900,eebb9702,"India, World Bank sign financing agreement for...",http://www.abplive.in/business/india-world-ban...,NEWS WB- All Streams,NaN,2017-12-21 09:22:12,"<table border=""0"" cellspacing=""3"" cellpadding=...","New Delhi [India], Dec 20 (ANI): A financing a...","[institutes, india, skill, financing, training...",1031,en
4268,6832ce57,Rs 40000-crore development projects in limbo i...,http://www.moneycontrol.com/news/business/econ...,NEWS AIIB - All Streams,NaN,2017-12-10 09:40:00,"<table border=""0"" cellspacing=""3"" cellpadding=...","Development projects worth more than Rs 40,000...","[development, crore, andhra, eaps, state, proj...",4390,en
1663,30f8f65e,https://www.the-american-interest.com/2018/01/...,https://www.the-american-interest.com/2018/01/...,NEWS AFDB- All Streams,NaN,2018-01-03 12:21:54,"<table border=""0"" cellspacing=""3"" cellpadding=...",Ten Lessons\n\nDevelopment with Chinese Charac...,"[transitions, university, chinese, united, dev...",575,en
3789,5ec16472,$300 Million to Expand and Upgrade Infrastruct...,https://jis.gov.jm/300-million-to-expand-and-u...,NEWS IDB - All Streams,NaN,2018-04-27 16:28:43,"<table border=""0"" cellspacing=""3"" cellpadding=...","Minister of Industry, Commerce, Agriculture an...","[research, development, agriculture, fisheries...",2621,en
6657,989c9942,ADB Provides $346 Million to Upgrade State Hig...,http://www.business-standard.com/article/news-...,NEWS ADB - All Streams,NaN,2017-12-10 01:00:00,"<table border=""0"" cellspacing=""3"" cellpadding=...",The Asian Development Bank's (ADB) Board of Di...,"[highways, improvement, india, state, upgrade,...",2532,en


In [10]:
bankNewsData = pd.merge(bankLabel,newsData[['article_id','article_text','article_keywords']], on = 'article_id')

---------------

# Bank Tags - One possible direction

1. we can extract banks from article text, article key words,feedlabel to see if bank title is in title, artile text or article key words. create binary feature out of this

2. cross check which binary feature is most predictive of bank tags and use this as a heuristic rules to tag all the news feed

3. manually sanity check the tag results


In [50]:
bankNames = bankNewsData['Bank1'].unique()
bankNames

array(['adb', 'afdb', 'aiib', 'ebrd', 'eib', 'idb', 'wb'], dtype=object)

   try build a function to iterate through  the title and bank names to check if bank key words are there

In [51]:
'adb' in bankNewsData['title'].iloc[0].lower().split()

True

In [17]:
'adb' in bankNewsData['article_keywords'].iloc[0]

True

In [19]:
bankNewsData['feed_label'].iloc[0]

'NEWS ADB - All Streams'

In [24]:
'adb' in bankNewsData['feed_label'].iloc[0].lower().split(' ')

True

### this is just simple keyword matching and we can be a lot more creative, the next step is to check if when any of the binary feature is true, what percentage of it is properly assigned ###

--------------------

# Country Tags 

**Simple demo **

country code names : http://www.geonames.org/countries/

In [5]:
from geotext import GeoText


In [21]:
for t, p in zip(newsData.title.head(), newsData.article_text.head()):
    places_title, places_text = GeoText(t), GeoText(p)
    print( t , places_title.country_mentions, places_text.country_mentions)

India, World Bank sign financing agreement for skill enhancement project OrderedDict([('IN', 1)]) OrderedDict([('IN', 5)])
Rs 40000-crore development projects in limbo in Andhra OrderedDict() OrderedDict([('TR', 1), ('IN', 1)])
https://www.the-american-interest.com/2018/01/03/development-chinese-characteristics/ OrderedDict() OrderedDict([('CN', 2)])
$300 Million to Expand and Upgrade Infrastructure at Bodles Research Station OrderedDict() OrderedDict([('JM', 4)])
ADB Provides $346 Million to Upgrade State Highways in Karnataka, India OrderedDict([('IN', 1)]) OrderedDict([('IN', 2)])


In [19]:
print(newsData.iloc[3].article_text)

Minister of Industry, Commerce, Agriculture and Fisheries, Hon. Audley Shaw . + - Photo: Dave Reid Minister of Industry, Commerce, Agriculture and Fisheries, Hon. Audley Shaw . Story Highlights The Government has allocated some $300 million towards the Bodles Research Station redevelopment programme.

The funds are to be used to expand and upgrade the infrastructure of the facility located in Old Harbour, St. Catherine.

Minister of Industry, Commerce, Agriculture and Fisheries, Hon. Audley Shaw, made the disclosure at a signing ceremony for a memorandum of understanding (MOU) between the Ministry and the Caribbean Agricultural Research and Development Institute (CARDI) for the refurbishing of the Bodles Research Station’s seed laboratory and storage facility on April 25, at the Ministry in New Kingston.



The Government has allocated some $300 million towards the Bodles Research Station redevelopment programme.

The funds are to be used to expand and upgrade the infrastructure of the

In [22]:
countryLabels = pd.read_csv('../Data/Labeled_Data/countries.csv')

In [23]:
countryLabels.head()

,article_id,published,title,url,feed_label,Country1
0,10f9ed2,2018-01-11,ADB Provides Support for Three Infrastructure ...,http://moderndiplomacy.eu/2018/01/11/adb-provi...,NEWS ADB - All Streams,cambodia
1,c0eece9b,2018-05-13,ADB Helps Inaugurate New Power Distribution Ne...,http://feedproxy.google.com/~r/adb_news/~3/2My...,NEWS ADB - All Streams,afghanistan
2,d1d79dd8,2018-02-20,ADB Provides $360 Million for Rolling Stock to...,http://feedproxy.google.com/~r/adb_news/~3/v9s...,NEWS ADB - All Streams,bangladesh
3,f0d65e5,2018-02-25,ADB provides financing to Thailand's B.Grimm P...,https://www.dealstreetasia.com/stories/adb-b-g...,NEWS ADB - All Streams,thailand
4,4a557358,2018-02-26,ADB's $235m loan to support B.Grimm Power expa...,https://www.power-technology.com/news/adbs-235...,NEWS ADB - All Streams,thailand


# Sector Tags 

This is a tougher one .. we could use the project description information and the sector tags from the Projects data - and try to transfer a model to the news articles. 

Or we could try to apply some other model or logic to derive the sectors. 


See the sector information here : https://docs.google.com/document/d/1bU45HW4wavUKkrd0mzGzzrwTKUgejNgL-gW_AS465IU/edit



In [24]:
sectorLabels = pd.read_csv('../Data/Labeled_Data/sectors.csv')

In [25]:
sectorLabels.head()

,article_id,published,title,url,feed_label,Sectors,cl_Sector,top_sector
0,10f9ed2,2018-01-11,ADB Provides Support for Three Infrastructure ...,http://moderndiplomacy.eu/2018/01/11/adb-provi...,NEWS ADB - All Streams,Infrastructure,infrastructure,infrastructure
1,c0eece9b,2018-05-13,ADB Helps Inaugurate New Power Distribution Ne...,http://feedproxy.google.com/~r/adb_news/~3/2My...,NEWS ADB - All Streams,Energy,energy,energy
2,d1d79dd8,2018-02-20,ADB Provides $360 Million for Rolling Stock to...,http://feedproxy.google.com/~r/adb_news/~3/v9s...,NEWS ADB - All Streams,Transport,transport,transport
3,f0d65e5,2018-02-25,ADB provides financing to Thailand's B.Grimm P...,https://www.dealstreetasia.com/stories/adb-b-g...,NEWS ADB - All Streams,"Construction, Finance","construction, finance",construction
4,4a557358,2018-02-26,ADB's $235m loan to support B.Grimm Power expa...,https://www.power-technology.com/news/adbs-235...,NEWS ADB - All Streams,"Construction, Finance","construction, finance",construction
